<a href="https://colab.research.google.com/github/Qasimrizvi1245/CSProjects/blob/main/Copy_of_GenerateDistricts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

-----
This Notebook contains the implementation of district creation
-----
-----

In [ ]:
%pip install pandas
%pip install geojson
%pip install shapely
%pip install PyShp
%pip install shapely

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\laser\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\laser\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\laser\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\laser\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\laser\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [ ]:
import pandas as pd
import shapefile as shp
from shapely.geometry import Polygon,shape,MultiPolygon
from shapely.ops import unary_union
import time, math


### Loading Base Map Precincts and Contiguity List

In [ ]:
#assignment will contain the precinct-district assignments from the given base map
assignment = pd.read_csv('map_export_data/precinct-assignments.csv', index_col=0)

#precinct_list=[]
#for p in range(0,len(assignment)):
#    precinct_list.append(assignment.iloc[p].GEOID20)

#contiguity_list will contain the list of contiguous precincts
contiguity_list = pd.read_csv('Contiguity/Contiguity.csv', converters={1: pd.eval}, index_col=0)

#pre_data contains data needed for decision-making
pre_data = pd.read_csv('final_NJ_2020_State_Legislative_data.csv')


#load shape data
shpfile = 'nj_vtd_2020_bound/nj_vtd_2020_bound.shp'
dbffile = 'nj_vtd_2020_bound/nj_vtd_2020_bound.dbf'
shxfile = 'nj_vtd_2020_bound/nj_vtd_2020_bound.shx'
shpfile = shp.Reader(shp=shpfile, shx=shxfile, dbf=dbffile)

precinct_boundaries={}
count=0
for sr in shpfile.iterShapeRecords():
    geom = sr.shape # get geo bit
    rec = sr.record # get db fields
    precinct_boundaries[rec[3]]=geom
    count=count+1


print(len(contiguity_list))
print(assignment.head(5))
print(contiguity_list.head(5))
#print(assignment.District.loc['100'])



6754
             District
GEOID20              
34033020001         3
34033001501         3
34033042009         3
34033000703         3
34033042001         3
                                          Contiguous_Neighbors
Precinct                                                      
34033020001  [34033001501, 34033060E01, 34033020002, 340330...
34033001501  [34033020001, 34033042009, 34033060E01, 340336...
34033042009  [34033001501, 34033030001, 34033004204, 340330...
34033000703  [34033042001, 34033007001, 34033030001, 340330...
34033042001  [34033000703, 34033007001, 34033030001, 340330...


###Create functions for retrieving population and precinct data

In [ ]:
def population_district(district, assignment, pre_data):
    population = 0
    precincts = assignment.loc[assignment['District'] == district]
    for precinct in precincts.iterrows():
        if(pre_data.loc[pre_data.GEOID20==str(precinct[0]),'Total_2020_TotalAdj'].any()):
            population += pre_data.loc[pre_data.GEOID20==str(precinct[0]),'Total_2020_TotalAdj'].values[0]
    return population

In [ ]:
def population_precinct(precinct, pre_data):
    return pre_data.loc[pre_data.GEOID20==precinct,'Total_2020_TotalAdj'].values[0]

In [ ]:
def partisan_lean_district(district, assignment, pre_data):
    partisan_lean = 0
    precincts = assignment.loc[assignment['District'] == district]
    for precinct in precincts.iterrows():
        if(pre_data.loc[pre_data.GEOID20==str(precinct[0]),'Dem_2016-2021_Comp'].any()):
            partisan_lean += (pre_data.loc[pre_data.GEOID20==str(precinct[0]),'Dem_2016-2021_Comp'].values[0] 
                - pre_data.loc[pre_data.GEOID20==str(precinct[0]),'Rep_2016-2021_Comp'].values[0])
    return partisan_lean

In [ ]:
def partisan_lean_precinct(precinct, pre_data):
    return (pre_data.loc[pre_data.GEOID20==precinct,'Dem_2016-2021_Comp'].values[0] 
        - pre_data.loc[pre_data.GEOID20==precinct,'Rep_2016-2021_Comp'].values[0])


###Create data structures for districts (shape, population, partisanship, compactness)


In [ ]:
def pp_compactness(geom): # Polsby-Popper
    p = geom.length
    a = geom.area    
    return (4*math.pi*a)/(p*p)

In [ ]:
def create_district_compactness(district_shapes):
    district_compactness={}
    for i in range(1,41):
        district_compactness[i] = pp_compactness(district_shapes[i]);
    return district_compactness

In [ ]:
def create_district_populations(assignment, pre_data):
    district_populations={}
    for i in range(1,41):
        district_populations[i] = population_district(i, assignment, pre_data)
    return district_populations

In [ ]:
def create_district_partisan_leans(assignment, pre_data):
    district_partisan_leans={}
    for i in range(1,41):
        district_partisan_leans[i] = partisan_lean_district(i, assignment, pre_data)
    return district_partisan_leans

In [ ]:
def create_district_shapes(assignment, precinct_boundaries):
    count = 0
    district_shapes = {}
    district_list = {}

    for i in range(1,41):
        district_list[i] = []
    for p in assignment.index:
        d = assignment.loc[p].iloc[0]
        if (len(p) == 11):
            try:
                district_list[d].append(Polygon(shape(precinct_boundaries[p])))
            except:
                if (not p == '34003015016'):
                    #exclude the broken precinct
                    district_list[d].append(shape(precinct_boundaries[p]))
                continue
            count = count + 1
    print(f"Ok, got {count} precinct shapes from data file")

    for i in range(1,41):
        district_shapes[i] = unary_union(district_list[i])

    for i in range(1,41):
        if (isinstance(district_shapes[i], MultiPolygon)):
            print("ERROR, shape combination did not work")
            print(i)
    return district_shapes

### Logic for swapping two districts


In [ ]:
def swap_district(p, district, assignment, district_shapes, district_populations, district_partisan_leans, district_compactness, precinct_boundaries):
    
    #get district of p
    d = assignment.loc[p].iloc[0]
    

    ## Partisanship
    part_p = partisan_lean_precinct(p, pre_data)

    part_remove = district_partisan_leans[d]
    part_add = district_partisan_leans[district]
    
    #calculate the absolute partisanship before and after making the swap
    part_before = abs(part_remove) + abs(part_add)
    part_after = abs(part_remove - part_p) + abs(part_add + part_p)

    #if partisanship remains the same or increases don't swap
    if (part_after >= part_before):
        return False


    ## Population
    #compute population variance check
    tolerance = 0.04
    pop_p = population_precinct(p, pre_data)

    pop_remove = district_populations[d] - pop_p
    pop_add = district_populations[district] + pop_p

    #if the variance is too high, break (saves computation)
    if ((pop_remove < (232075*(1-tolerance))) or (pop_add > (232075*(1+tolerance)))):
        return False



    ## Contiguity / Polsby-Popper Compactness

    #remove the precinct from one district and add it to another
    shapeRemove = district_shapes[d].symmetric_difference(Polygon(shape(precinct_boundaries[p])))
    shapeAdd = unary_union([district_shapes[district], Polygon(shape(precinct_boundaries[p]))])

    #Ensure compactness stays reasonable
    tolerance = 0.5
    pp_before = district_compactness[d] + district_compactness[district]
    pp_after = pp_compactness(shapeRemove) + pp_compactness(shapeAdd)

    if(pp_before > pp_after + tolerance):
        return False
    
    if (not isinstance(shapeRemove, MultiPolygon) and not isinstance(shapeAdd, MultiPolygon)):
        #update shape data structure
        district_shapes[d] = shapeRemove
        district_shapes[district] = shapeAdd

        #update population data structure
        district_populations[d] = pop_remove
        district_populations[district] = pop_add

        #update partisan data structure
        district_partisan_leans[d] = part_remove - part_p
        district_partisan_leans[district] = part_add + part_p


        #make the swap
        assignment.loc[p].iloc[0] = district
        return True
    return False

### Reassignment code (with n iterations)


In [ ]:
def reassign_precincts(assignment, district_shapes, district_populations, district_partisan_leans, district_compactness, contiguity_list, precinct_boundaries, iterations):
    #create set for visited 
    visited = set()

    for i in range(iterations):
        numAssignments = 0
        potSwaps = 0
        
        for p in assignment.index:    
            #if base point is visited previously, move on. Greatly reduces the number of matches in exchange for a most stable swap
            if (p in visited):
                #continue
                pass

            p_district = assignment.loc[p].iloc[0]
            try:
                neighbors = contiguity_list.loc[p].iloc[0]
            except:
                continue
            for n in neighbors:
                try:
                    n_district = assignment.loc[n].iloc[0]
                except:
                    continue
                if (n_district != p_district):# and (not (n in visited))):
                    potSwaps = potSwaps + 1
                    if (swap_district(n, p_district, assignment, district_shapes, district_populations, district_partisan_leans, district_compactness, precinct_boundaries)):
                        numAssignments = numAssignments + 1
                #mark this precinct as visited    
                visited.add(n)



        print(f"iteration: {i} swap% = {numAssignments*100/potSwaps} swaps: {numAssignments} potentialswaps: {potSwaps}")
        
        if (not numAssignments):
            break

        #write to intermediaries every 10 iterations to prevent against crashes
        #algorithm is deterministic, so can pick up again after a crash
        if ((i) % 10 == 0):
            timestr = time.strftime("%Y%m%d-%H%M%S")
            print(f"writing iteration {i} to intermediaries...")
            assignment.to_csv('intermediaries/' + timestr + '_assignment.csv')
            print(f"Done!")
        visited.clear()

    #test for noncontiguous districts
    for i in range(1,41):
        if (isinstance(district_shapes[i], MultiPolygon)):
            print("ERROR, shape combination did not work")
            print(i)
    
    #convert assignment to csv
    print(f"writing completed assignment to final/ ...")
    timestr = time.strftime("%Y%m%d-%H%M%S")
    assignment.to_csv('final/' + timestr + '_assignment.csv')


### After initializing above, run this script to generate the precinct assignments

In [ ]:
#number of iterations
n = 20

#if the program crashes, look at the latest intermediary in /intermediaries and load it as the base assignment
print(f"Loading base assignment...")
#assignment = pd.read_csv('map_export_data/precinct-assignments.csv', index_col=0)
assignment = pd.read_csv('base/most_proportional.csv', index_col=0)
print(f"Done!")

print(f"Creating district shapes data structure...")
district_shapes = create_district_shapes(assignment, precinct_boundaries)
print(f"Done!")

print(f"Creating district populations data structure...")
district_populations = create_district_populations(assignment, pre_data)
print(f"Done!")

print(f"Creating district partisan leans data structure...")
district_partisan_leans = create_district_partisan_leans(assignment, pre_data)
print(f"Done!")

print(f"Creating district compactness data structure...")
district_compactness = create_district_compactness(district_shapes)
print(f"Done!")

print(f"Assigning precincts with n={n} iterations...")
reassign_precincts(assignment, district_shapes, district_populations, district_partisan_leans, district_compactness, contiguity_list, precinct_boundaries, n)
print(f"Done!")

Loading base assignment...
Done!
Creating district shapes data structure...


C:\Users\laser\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\shapely\geometry\polygon.py:539: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  ret = geos_linearring_from_py(shell)
C:\Users\laser\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\shapely\geometry\polygon.py:539: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  ret = geos_linearring_from_py(shell)


Ok, got 6342 precinct shapes from data file
Done!
Creating district populations data structure...
Done!
Creating district partisan leans data structure...
Done!
Creating district compactness data structure...
Done!
Assigning precincts with n=20 iterations...
iteration: 0 swap% = 4.541734860883797 swaps: 222 potentialswaps: 4888
writing iteration 0 to intermediaries...
Done!
iteration: 1 swap% = 4.0673211781206176 swaps: 232 potentialswaps: 5704
iteration: 2 swap% = 1.9056847545219637 swaps: 118 potentialswaps: 6192
iteration: 3 swap% = 0.529925187032419 swaps: 34 potentialswaps: 6416
iteration: 4 swap% = 0.339715873996294 swaps: 22 potentialswaps: 6476
iteration: 5 swap% = 0.07687576875768758 swaps: 5 potentialswaps: 6504
iteration: 6 swap% = 0.07678132678132678 swaps: 5 potentialswaps: 6512
iteration: 7 swap% = 0.10705000764642912 swaps: 7 potentialswaps: 6539
iteration: 8 swap% = 0.06093845216331505 swaps: 4 potentialswaps: 6564
iteration: 9 swap% = 0.0453857791225416 swaps: 3 potent